In [1]:
import time

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options 
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd
from bs4 import BeautifulSoup
import requests

#contentsArea > div.recruitment-notice > div.notice-info > div > ul:nth-child(1) > li > span.value 주소
#contentsArea > div.recruitment-notice > div.notice-info > div > ul:nth-child(2) > li:nth-child(1) > span.value 주택유형
#contentsArea > div.recruitment-notice > div.notice-info > div > ul:nth-child(2) > li:nth-child(2) > span.value 세대수
#contentsArea > div.recruitment-notice > div.notice-info > div > ul:nth-child(2) > li:nth-child(3) > span.value 임대사업자
#basicInfo > div > div > table > tbody > tr:nth-child(13) > td 주차대수
#suplySwipeBody > div.swiper-slide.swiper-slide-active > div:nth-child(1) > div:nth-child(1) > span 무슨 숫자
#suplySwipeBody > div.swiper-slide.swiper-slide-active > div:nth-child(1) > div:nth-child(2) > div:nth-child(1) > span.info-value 공급면적(전용)
#suplySwipeBody > div.swiper-slide.swiper-slide-active > div:nth-child(1) > div:nth-child(2) > div:nth-child(2) > span.info-value 공급면적(공용)
#suplySwipeBody > div.swiper-slide.swiper-slide-active > div:nth-child(2) > div > div > div > dl.total > dd 임대보증금(원)
#suplySwipeBody > div.swiper-slide.swiper-slide-active > div:nth-child(2) > div > div > div > dl:nth-child(2) > dd 월 입대료(원)
#suplySwipeBody > div.swiper-slide.swiper-slide-active > div:nth-child(2) > div > div > div > dl:nth-child(3) > dd 전환 보증금(원)
#detailArea > div > ul > li:nth-child(3) > div > div > div.swiper-bottom-wrap > div.swiper-button-next 다음 버튼



options = webdriver.ChromeOptions()
options.add_argument("--ignore-local-proxy")

#크롬드라이버 매니저를 통해 최신 드라이버를 설치하고 서비스 만들어냄
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

URL='https://www.myhome.go.kr/hws/portal/sch/selectRentalHouseInfoListView.do'
driver.get(url=URL)
#여기까지는 그냥 기본 틀 가져옴옴
time.sleep(3)

#팝업닫기기
opens = driver.find_element(By.CSS_SELECTOR,f'body > div > aside > div.quick-menu.open > div > button > i')
opens.click()

#펼치기
opens = driver.find_element(By.CSS_SELECTOR,f'#contentsArea > div.housing-searchbox > button > i')
opens.click()
#data2는 한번에 저장할려고 일단 만들어만 놔서 오류 없을 때 신경쓰시면 될거 같아요요
data2 = []
#원래 1,26이어야 하는데 오류만 확인하려고 이렇게함 구가 25개인거 같아요요
for i in range(22,23) :
    #판다스에 넣을 리스트
    data = []
    #구 선택
    where = driver.find_element(By.CSS_SELECTOR,f'#signguCode > option:nth-child({i})')
    where.click()
    time.sleep(3)
    #검색하기 선택
    search = driver.find_element(By.CSS_SELECTOR,f'#frm > div > div.btn-wrap > div > button:nth-child(2)')
    search.click()
    time.sleep(3)
    count = 4
    em = False

    for t in range(100) :

        #한 페이지에 리스트가 5개 나와서 이렇게 함함
        for j in range(1,6) :
            try:
                #페이지 열기
                #schTbody > tr:nth-child(2) > td:nth-child(1)
                page = driver.find_element(By.CSS_SELECTOR,f'#schTbody > tr:nth-child({j}) > td:nth-child(1) > a')
                page.click()
                time.sleep(3)

                #페이지를 한번에 정적 크롤링으로 읽어서 데이터 처리리
                html = driver.page_source
                soup = BeautifulSoup(html, 'html.parser')
                aptName = soup.select_one(f'#contentsArea > div.recruitment-notice > div.notice-head > div.title-wrap > p').text.strip()
                address = soup.select_one(f'#contentsArea > div.recruitment-notice > div.notice-info > div > ul:nth-child(1) > li > span.value').text.strip()
                homeKind = soup.select_one(f'#contentsArea > div.recruitment-notice > div.notice-info > div > ul:nth-child(2) > li:nth-child(1) > span.value').text.strip()
                homeCount = soup.select_one(f'#contentsArea > div.recruitment-notice > div.notice-info > div > ul:nth-child(2) > li:nth-child(2) > span.value').text.strip()
                co = soup.select_one(f'#contentsArea > div.recruitment-notice > div.notice-info > div > ul:nth-child(2) > li:nth-child(3) > span.value').text.strip()
                car = soup.select_one(f'#basicInfo > div > div > table > tbody > tr:nth-child(13) > td').text.strip()

                # # 데이터를 리스트에 추가
                # data.append({
                #     "주소": address,
                #     "주택유형": homeKind,
                #     "세대수": homeCount,
                #     "임대사업자": co,
                #     "주차대수": car
                # })

                #원래 이미지 까지하려했는데 지금은 일단 위에거 부터 하려고 아직 추가 안함
                images = soup.select('#imageSwipeBody img')

                # src 속성 추출
                image_urls = [img['src'] for img in images]
                # 앞에 추가할 문자열
                base_url = "https://www.myhome.go.kr"

                # 각 URL 앞에 문자열 추가 후 연결
                result = "\t".join([base_url + url for url in image_urls])

                #얼마인지 금액나오는 부분 다음으로 넘어가기 안되서 이렇게 한번에 가져와서 처리리
                no = soup.select_one(f'#suplySwipeBody').get_text(separator="\n")

                lines = no.split("\n")

                # 데이터를 담을 리스트 초기화
                item = {}  # 개별 항목 저장

                for idx, line in enumerate(lines):
                    line = line.strip()  # 공백 제거
                    if line.isdigit() and line != "0":  # 숫자로만 이루어진 줄은 공급번호로 간주
                        if item:  # 이전 데이터가 있다면 저장
                            # 기존 데이터와 중복되지 않을 때만 추가
                            if item not in data:
                                # 주소, 주택유형 등 추가
                                item.update({
                                    "단지명": aptName,
                                    "주소": address,
                                    "주택유형": homeKind,
                                    "세대수": homeCount,
                                    "임대사업자": co,
                                    "주차대수": car,
                                    "img" : result
                                })
                                data.append(item)
                        # 새 항목 초기화
                        item = {"공급번호": line}
                    elif "공급면적(전용)" in line:
                        item["공급면적(전용)"] = lines[idx + 1].strip() if idx + 1 < len(lines) else None
                    elif "공급면적(공용)" in line:
                        item["공급면적(공용)"] = lines[idx + 1].strip() if idx + 1 < len(lines) else None
                    elif "임대보증금(원)" in line:
                        item["임대보증금(원)"] = lines[idx + 1].strip() if idx + 1 < len(lines) else None
                    elif "월 임대료(원)" in line:
                        item["월 임대료(원)"] = lines[idx + 1].strip() if idx + 1 < len(lines) else None
                    elif "전환보증금(원)" in line:
                        item["전환보증금(원)"] = lines[idx + 1].strip() if idx + 1 < len(lines) else None

                # 마지막 항목 저장
                if item and item not in data:
                    # 주소, 주택유형 등 추가
                    item.update({
                        "단지명": aptName,
                        "주소": address,
                        "주택유형": homeKind,
                        "세대수": homeCount,
                        "임대사업자": co,
                        "주차대수": car,
                        "img" : result
                    })
                    data.append(item)
                    data2.append(item)
            
                
                #목록
                back = driver.find_element(By.CSS_SELECTOR,f'#contentsArea > div.bottom-btn-wrap > button')
                back.click()
                time.sleep(3)
            except Exception as e:
                print(f"{e}")
                em = True
                break
        if em == True :
            time.sleep(3)
            break
        try :
            lst = driver.find_element(By.CSS_SELECTOR,f'#pageDiv > ul > li:nth-child({count}) > a')
            lst.click()
            time.sleep(3)
            count = count + 1
            if count == 9 :
                count = 4            
        except Exception as e:
            print(f"{e}")
            break
    #pageDiv > ul > li:nth-child(4) > a
    #팝업닫기
    opens = driver.find_element(By.CSS_SELECTOR,f'body > div > aside > div.quick-menu.open > div > button > i')
    opens.click()

    #펼치기
    opens = driver.find_element(By.CSS_SELECTOR,f'#contentsArea > div.housing-searchbox > button > i')
    opens.click()
    df = pd.DataFrame(data)

    # CSV 파일로 저장
    df.to_csv(f'parsed_data{i}.csv', index=False, encoding='utf-8-sig')
    
df2 = pd.DataFrame(data2)

    # CSV 파일로 저장
df2.to_csv(f'parsed_data최종{i}.csv', index=False, encoding='utf-8-sig')

Message: no such element: Unable to locate element: {"method":"css selector","selector":"#schTbody > tr:nth-child(5) > td:nth-child(1) > a"}
  (Session info: chrome=131.0.6778.265); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00A7FD53+23747]
	(No symbol) [0x00A07D54]
	(No symbol) [0x008DBE53]
	(No symbol) [0x0091FCA6]
	(No symbol) [0x0091FEEB]
	(No symbol) [0x0095D852]
	(No symbol) [0x00941E44]
	(No symbol) [0x0095B41E]
	(No symbol) [0x00941B96]
	(No symbol) [0x00913F3C]
	(No symbol) [0x00914EBD]
	GetHandleVerifier [0x00D5AC73+3017699]
	GetHandleVerifier [0x00D6B93B+3086507]
	GetHandleVerifier [0x00D640F2+3055714]
	GetHandleVerifier [0x00B15AF0+637536]
	(No symbol) [0x00A10A5D]
	(No symbol) [0x00A0DA28]
	(No symbol) [0x00A0DBC5]
	(No symbol) [0x00A007F0]
	BaseThreadInitThunk [0x76A07BA9+25]
	RtlInitializeExceptionChain [0x77C3C0CB+107]
	RtlClearBits [0x